Define your variables (That's the only cells you need to change). Then run the whole notebook

In [7]:
method = 'Rigid'
path_to_data = '/Users/aichah/Desktop/TLI/data'
save_path = '/Users/aichah/Desktop/TLI/data'
group_by = '210218'
ch_names = ['GFP', 'red']
save_name = '210218_LP40_P36'
compile = True
metric = 'CC'

In [ ]:
if save_path[-1] != '/':
    save_path += '/'

Load packages and define functions

In [8]:
import os
from timeit import default_timer as timer

In [1]:
import utils.datautils as datautils
import utils.registration as registration

ImportError: dlopen(/Users/aichah/opt/miniconda3/envs/growth/lib/python3.9/site-packages/ants/lib/addNoiseToImage.cpython-39-darwin.so, 2): Library not loaded: /opt/X11/lib/libpng16.16.dylib
  Referenced from: /Users/aichah/opt/miniconda3/envs/growth/lib/python3.9/site-packages/ants/lib/addNoiseToImage.cpython-39-darwin.so
  Reason: image not found

Running the pipeline

In [4]:
# reading list of 3D images into dictionary of channels as indicated in ch_names, and applying median filter on the last ch
image_4D = datautils.read_files(path_to_data, group_by ,compile=compile, ch_names=ch_names, order=False,
                                save=False)

compiling_files:  10%|█         | 1/10 [00:20<03:04, 20.55s/it]

split_convert runtime 20.408579634999995


compiling_files:  20%|██        | 2/10 [00:40<02:43, 20.42s/it]

split_convert runtime 20.213132924999996


compiling_files:  30%|███       | 3/10 [01:01<02:22, 20.37s/it]

...

compiling_files:  90%|█████████ | 9/10 [02:59<00:19, 19.73s/it]

split_convert runtime 19.466358325000016


compiling_files: 100%|██████████| 10/10 [03:19<00:00, 19.94s/it]

split_convert runtime 19.41588670599998
dict_keys(['GFP', 'red']) <class 'list'> 10
compiling the GFP channel


compiling the red channel
files_to_4D runtime 200.849550234


In [10]:
# measuring the cosine_similarity check of GFP channel before registration
pre_checks = datautils.similarity_4D(image_4D[ch_names[0]], save=True, save_path=save_path, save_file='pre_checks.csv')

cosine_sim for timepoint: 100%|██████████| 9/9 [00:06<00:00,  1.44it/s]

worked
finished measuring similarity check 6.240511722000008


In [17]:
# running antspy, saving output into temp 3D files, and then compiling them to 4D and finally deleting the temp 3D files
for method in ants_method:
    start_time = timer()
    name_ch1 = method+'_'+ch_names[0]+'_'
    name_ch2 = method+'_'+ch_names[1]+'_'
    registration.antspy_drift_corr(img_4D_g=image_4D[ch_names[0]],
                                    img_4D_r=image_4D[ch_names[1]], 
                                    ch_names=ch_names,
                                    save_path=save_path, 
                                    save_name=save_name,
                                    ref_t = int(len(image_4D[ch_names[0]])/2),
                                    drift_corr=method)

    # compiling the registered 3D images into 4D and deleting the temp files for both channels 
    gfp_list = datautils.get_file_names(path=save_path, group_by=ch_names[0]+'_'+method+save_name, order=False, 
                                        nested_files=False, criteria='tif')
    gfp_name = ch_names[0]+'_'+method+'_4D_'+group_by+'.tif'
    gfp_4D = datautils.files_to_4D(files_list=gfp_list, ch_names=[ch_names[0]], filter=False,
                                    save=True, save_path=save_path, save_file=gfp_name,
                                    xy_pixel=0.0764616, z_pixel=0.4)
    for file in gfp_list:
        os.remove(file)

    red_list = datautils.get_file_names(path=save_path, group_by=ch_names[1]+'_'+method+save_name, order=False, 
                                nested_files=False, criteria='tif')
    red_name = ch_names[1]+'_'+method+'_4D_'+group_by+'.tif'
    red_4D = datautils.files_to_4D(files_list=red_list, ch_names=[ch_names[1]], filter=False,
                                    save=True, save_path=save_path, save_file=red_name,
                                    xy_pixel=0.0764616, z_pixel=0.4)
    for file in red_list:
        os.remove(file)
    
    # measuring the cosine_similarity check of GFP channel after registration
    post_checks = datautils.similarity_4D(gfp_4D, save=True, 
                                            save_path=save_path, 
                                            save_file=method+'_checks.csv')

    # deleting the final compiled 4D images
    del gfp_4D, red_4D 

    print(method, ' is done, and it took', timer()-start_time)   

  0%|          | 0/11 [00:00<?, ?it/s]


AttributeError: module 'ants' has no attribute 'from_numpy'

In [ ]:
print('memory cleared, and kernel stopped')
print('Job is DONE!')
os._exit(00)